In [1]:
import os
# Do! 프로젝트 root 경로로 설정
project_path = "D:/Users\workspace/jeju-bus-stations-clustering_MH"
os.chdir(project_path)

In [2]:
from multiprocessing import Pool
import multiprocessing
import pandas as pd
import numpy as np
import datetime

import bus.analyzer as anz

In [3]:
def parallelize_dataframe(func, df, core = multiprocessing.cpu_count()-4):
    df_split = np.array_split(df, core)
    pool = Pool(core)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [4]:
# 데이터 기간 설정
start_date = datetime.datetime(2019, 6, 1)
end_date = datetime.datetime(2019, 8, 29)

# 로딩할 파일 명 리스트 생성
input_path_list = anz.make_input_path(start_date, end_date)

usage_df = parallelize_dataframe(anz.load_total_usage_data, input_path_list, core = 10)

cluster_df = anz.load_cluster_df()

In [5]:
usage_selector = ['user_id', 'geton_datetime', 'geton_station_id', 'getoff_datetime', 'getoff_station_id']
usage_df = usage_df[usage_selector]

cluster_selector = ['station_id', 'cluster_group', 'cluster_target']
cluster_df = cluster_df[cluster_selector]

geton_columns = {}
getoff_columns = {}
for column in cluster_selector:
    geton_columns[column] = "geton_"+column
    getoff_columns[column] = "getoff_"+column
geton_cluster_df = cluster_df.rename(columns=geton_columns)
getoff_cluster_df = cluster_df.rename(columns=getoff_columns)

merged_df = usage_df
merged_df = pd.merge(merged_df, geton_cluster_df, on=['geton_station_id'])
merged_df = pd.merge(merged_df, getoff_cluster_df, on=['getoff_station_id'])
merged_selector = ['user_id', 'geton_datetime', 'geton_cluster_group', 'geton_cluster_target', 'getoff_datetime', 'getoff_station_id']
merged_df

,user_id,geton_datetime,geton_station_id,getoff_datetime,getoff_station_id,geton_cluster_group,geton_cluster_target,getoff_cluster_group,getoff_cluster_target
0,f6f372cf8c6732eafc2a82b4f9d7a08bb3b493213ea4ef...,2019-06-01 15:02:25,988,2019-06-01 15:16:38,26.0,제주시,49,제주시,52
1,3f6cfe2429f6e8d62cddc39222993405789b7b9ea2a592...,2019-06-01 15:02:29,988,2019-06-01 15:16:40,26.0,제주시,49,제주시,52
2,eb42f68c2df19467202dfe5b54ee75ef31ca7f9d260809...,2019-06-03 07:31:23,988,2019-06-03 07:49:31,26.0,제주시,49,제주시,52
3,eb42f68c2df19467202dfe5b54ee75ef31ca7f9d260809...,2019-06-04 07:32:41,988,2019-06-04 07:50:06,26.0,제주시,49,제주시,52
4,eb42f68c2df19467202dfe5b54ee75ef31ca7f9d260809...,2019-06-05 07:31:04,988,2019-06-05 07:47:44,26.0,제주시,49,제주시,52
...,...,...,...,...,...,...,...,...,...
7903229,4ccf33ce2515b11c1b795575b3b683b828c537f7ebd6e9...,2019-06-15 09:25:33,2167,2019-06-15 09:40:11,2171.0,서귀포시,34,서귀포시,441
7903230,4ccf33ce2515b11c1b795575b3b683b828c537f7ebd6e9...,2019-06-22 11:42:07,2167,2019-06-22 11:56:45,2171.0,서귀포시,34,서귀포시,441
7903231,56981d1ee66f71a8e951f2ca5007a1883d4e905d237ea0...,2019-08-21 13:09:23,1401,2019-08-21 15:33:20,1406.0,제주시,637,제주시,744
7903232,a4a1b9cc64878611cd37184a815c7309e8030cd2da56c4...,2019-08-07 10:25:10,2861,2019-08-07 10:26:24,2861.0,서귀포시,510,서귀포시,510
